# Group Chat Analysis System
Merupakan suatu sistem untuk menganalisa suatu pesan di dalam group apakah termasuk pesan yang baik, buruk, benar, atau hoax. Dengan analisa tersebut, nantinya dapat disimpulkan bahwa semakin tinggi dan banyak pesan yang teranalisa buruk atau hoax maka group tersebut dapat dipastikan adalah group yang tidak benar. Dan sebaliknya, jika di dalam grup tersebut banyak pesan baik dan informasi yang disampaikan adalah benar, maka dapat disimpulkan bahwa group tersebut adalah group yang baik.

Sistem ini disusun oleh

- Fitri Amalia Langgundi
- Khoerul Umam
- Muhammad Ikhsan Hadi
- Rozak Ilham Aditya

Dalam mata kuliah : Sistem Pakar

### Group Chat Analysis 
Inisalisasi library yang dibutuhkan. Beberapa library yang digunakan diantaranya, library untuk memfilter pesan, library untuk mengolah link, library untuk mengolah dataset, library untuk menentukan tingkat karakteristik pesan, dan library lain akan ditambahkan seiring dengan bertambahnya kompleksitas sistem

In [1]:
import re
import tldextract
import pandas as pd
import jellyfish as jelly
import numpy as np

#### Inisialisai Class
Class yang digunakan diberi nama class Pakar. Di dalamnya terdapat berbagai fungsi yang akan memproses data pesan sehingga dapat diketahui karakteristik pesan tersebut. Saat ini sistem hanya dapat memproses satu per satu pesan.

In [25]:
class Pakar:
    def __init__ (self):
        self.text = ''
        self.link = []
        self.domaindataset = pd.read_csv('D:\Project\Self\Pakar\domain.csv', skiprows=0)
        self.result = []

    def process(self, text):
        self.text = text
        self.checkLink()
        self.checkText()

    def checkLink(self):
        regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
        url = re.findall(regex,self.text)       
        for x in url:
            link = x[0]
            domain = tldextract.extract(x[0])
            suffix = domain.suffix

            if (domain.suffix == ''):
                suffix = domain.domain
            
            checkDomain = self.domaindataset.loc[self.domaindataset['Domain'] == '.' + suffix]
                
            if (link.find('https') != -1):
                # if self.domaindataset['Domain'].str.contains(suffix).any():
                if (checkDomain.empty == False):
                    self.link.append([link, 'Aman', checkDomain['Domain'].values, checkDomain['Type'].values, checkDomain['Sponsoring Organisation'].values])
                    # np.append([link, 'safe'], self.link)
                else:
                    # np.append([link, 'sus'], self.link)
                    self.link.append([link, 'Mencurigakan', checkDomain['Domain'].values, checkDomain['Type'].values, checkDomain['Sponsoring Organisation'].values])
            else:
                # if self.domaindataset['Domain'].str.contains(suffix).any():
                if (checkDomain.empty == False):
                    # np.append([link, 'low security'], self.link)
                    self.link.append([link, 'Keamanan Lemah', checkDomain['Domain'].values, checkDomain['Type'].values, checkDomain['Sponsoring Organisation'].values])
                else:
                    self.link.append([link, 'Bahaya', checkDomain['Domain'].values, checkDomain['Type'].values, checkDomain['Sponsoring Organisation'].values])
                    # np.append([link, 'danger'], self.link)

    def checkText(self):
        dataset = pd.read_csv('D:\Project\Self\Pakar\hoax.csv', header=None)
        for index, row in dataset.iterrows():
            self.result.append([index, jelly.jaro_distance(self.text, row[0])])
            # np.append([index, jelly.levenshtein_distance(self.text, row[0])], self.result)
        self.result = sorted(self.result, key=lambda x: x[1], reverse=True)
    
    def get_domain_dt(self):
        return self.domaindataset
    
    def get_result(self):
        return self.result

    def show_result(self):
        print('Tingkat ke-hoax-an pesan: ', self.result[0][1], '\n')
        if (self.link != None):
            for x in self.link:
                print('Link: ', x[0], ' terindikasi ', x[1], '\n')
                print('Domain: ', x[2], '\n')
                print('Tipe Link: ', x[3], '\n')
                print('Keterangan: ', x[4], '\n\n')




### Testing
Testing digunakan per pesan. Dilakukan seperti di bawah ini, pesan akan diketahui seberapa dekat karakteristik dengan dataset yang digunakan, termasuk jika ada link di dalamnya, akan di ketahui link mana yang berbahaya dan link mana yang aman.

In [27]:
t = Pakar()
t.process('bang sate dapat dibeli di http://umam.keren dan di https://google.com')
t.show_result()

Tingkat ke-hoax-an pesan:  0.7004830917874396 

Link:  http://umam.keren  terindikasi  Bahaya 

Domain:  [] 

Tipe Link:  [] 

Keterangan:  [] 


Link:  https://google.com  terindikasi  Aman 

Domain:  ['.com'] 

Tipe Link:  ['generic'] 

Keterangan:  ['VeriSign Global Registry Services'] 


